In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dipy.core.sphere import disperse_charges, HemiSphere, Sphere
from dipy.direction import peak_directions
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.sims.voxel import multi_tensor, multi_tensor_odf, single_tensor
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import nibabel as nib
import random
import math

In [123]:
thetas = list(np.load("synthetic_data/thetas.npy"))
phis = list(np.load("synthetic_data/phis.npy"))

hemisphere = HemiSphere(theta=thetas, phi=phis) # We already dispersed charges when building the hemisphere in dipy_test
sphere = Sphere(xyz=np.vstack((hemisphere.vertices, -hemisphere.vertices)))


def detect_peaks(F, relative_peak_threshold, min_separation_angle):
    peak_format = np.zeros((len(F), 42))
    max_peak_count = 0
    for i, sample in enumerate(F):
        # Duplicate the sample for both hemispheres
        F_sphere = np.hstack((sample, sample)) / 2
        # Find peak directions
        directions, values, indices = peak_directions(F_sphere, sphere, relative_peak_threshold, min_separation_angle)
        directions = sample[indices][:, np.newaxis] * directions # multiplying with fractions
        directions_flattened = directions.flatten()
        peak_format[i][0:len(directions_flattened)] = directions_flattened
        if len(directions_flattened) / 3 > max_peak_count:
            max_peak_count = len(directions_flattened) / 3
    return peak_format, max_peak_count

In [124]:
class MatrixFactorizationNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MatrixFactorizationNet, self).__init__()
        # Define the network layers
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], output_size)
        self.softplus = nn.Softplus()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.softplus(self.fc4(x))  # Softplus activation for the output layer
        return x

class MatrixDataset(Dataset):
    def __init__(self, S, F):
        self.S = S
        self.F = F

    def __len__(self):
        return len(self.S)

    def __getitem__(self, idx):
        return self.S[idx], self.F[idx]
    
def apply_sparsity(F, sparsity_threshold=0.9):
    max_value, _ = F.max(dim = 1, keepdim = True)
    F[F < 0.1 * max_value] = 0
    return F


S = np.load("synthetic_data/S.npy")
F = np.load("synthetic_data/F.npy")
print(np.shape(S))
print(np.shape(F))

# Convert them to PyTorch tensors
S = torch.from_numpy(S).float()
# Separate into training and testing set
split = 0.9
num_entries = len(S)
train_len = int(num_entries * split)

S_train = S[:train_len]
F_train = F[:train_len]

S_test = S[train_len:]
F_test = F[train_len:]
batch_size = 1024

criterion = nn.MSELoss()

train_dataset = MatrixDataset(S_train, F_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MatrixDataset(S_test, F_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

N_test, _ = np.shape(S_test)
N, n_b = np.shape(S_train)
_, n = np.shape(F_train)

input_size = n_b
output_size = n

net = MatrixFactorizationNet(input_size, [512, 256, 128], output_size)
net.load_state_dict(torch.load("trained_model/model_15_12_custom_dist_1e5.pt", map_location=torch.device('cpu')))

(500000, 65)
(500000, 180)


<All keys matched successfully>

In [125]:
mask = nib.load('real_data/mask.nii').get_fdata().astype(bool)
white_mask = nib.load('real_data/white_matter_mask.nii').get_fdata().astype(bool)

hardi_snr10 = nib.load('real_data/DWIS_hardi-scheme_SNR-10.nii').get_fdata()
hardi_snr20 = nib.load('real_data/DWIS_hardi-scheme_SNR-20.nii').get_fdata()
hardi_snr30 = nib.load('real_data/DWIS_hardi-scheme_SNR-30.nii').get_fdata()

hardi_snr10_masked = hardi_snr10[mask]
hardi_snr20_masked = hardi_snr20[mask]
hardi_snr30_masked = hardi_snr30[mask]
hardi_snr10_masked *= (100/hardi_snr10_masked[0][0])
hardi_snr20_masked *= (100/hardi_snr20_masked[0][0])
hardi_snr30_masked *= (100/hardi_snr30_masked[0][0])

hardi_snr10_white_masked = hardi_snr10[white_mask]
hardi_snr20_white_masked = hardi_snr20[white_mask]
hardi_snr30_white_masked = hardi_snr30[white_mask]
hardi_snr10_white_masked *= (100/hardi_snr10_white_masked[0][0])
hardi_snr20_white_masked *= (100/hardi_snr20_white_masked[0][0])
hardi_snr30_white_masked *= (100/hardi_snr30_white_masked[0][0])

gt_white_masked = pd.read_pickle("real_data/ground_truth_peaks_white_masked.pkl")

In [126]:
test_set_hardi = torch.from_numpy(hardi_snr30_white_masked).float()
F_test_set_hardi = torch.from_numpy(np.zeros(hardi_snr30_white_masked.shape[0])).float()
test_dataset_hardi = MatrixDataset(test_set_hardi, F_test_set_hardi)
test_loader_hardi = DataLoader(test_dataset_hardi, batch_size=batch_size, shuffle=False)

In [127]:
with torch.no_grad():
    total_loss = 0.0
    F_pred_matrix_normalized_list = []

    for S_batch, F_batch in test_loader_hardi:
        S_batch_flattened = S_batch.to_dense().view(S_batch.size(0), -1)

        F_batch_pred = net(S_batch_flattened)
        F_batch_pred = apply_sparsity(F_batch_pred)
        F_batch_pred_normalized = F_batch_pred / F_batch_pred.sum(dim=1, keepdim=True)

        #loss = criterion(F_batch_pred_normalized, F_batch)

        #total_loss += loss.item()
        F_pred_matrix_normalized_list.append(F_batch_pred_normalized)

    # Calculate the average loss over all test samples
    avg_loss = total_loss / len(test_loader)
    print("Average Test Loss:", avg_loss)


Average Test Loss: 0.0


In [128]:
F_pred_np = np.concatenate([tensor.cpu().numpy().flatten() for tensor in F_pred_matrix_normalized_list])
F_pred_np = F_pred_np.reshape((-1,180)).astype(np.double)

In [8]:
indices_top10 = np.argpartition(F_pred_np[0], -10)[-10:]

# Get the top 10 largest elements
print(F_pred_np[0][indices_top10])
print(indices_top10)

indices_top3_true = np.argpartition(F_test[0], -3)[-3:]
# Get the top 3 largest elements
print(F_test[0][indices_top3_true])
print(indices_top3_true)

[0.         0.         0.         0.         0.04457189 0.08100978
 0.25074464 0.12433697 0.1849499  0.31438673]
[ 61  60  64  62  47  57 170  69 102 121]
[0. 0. 1.]
[ 60 179 132]


In [47]:
relative_peak_threshold = 0.1
min_separation_angle = 25

peak_format, peak_count = detect_peaks(F_pred_np[1].reshape(1,-1), relative_peak_threshold, min_separation_angle)

IndexError: Values in edges must be < len(odf)

In [10]:
peak_format

array([[0.30097467, 0.05795585, 0.06995984, ..., 0.        , 0.        ,
        0.        ],
       [0.22633837, 0.22264666, 0.24471267, ..., 0.        , 0.        ,
        0.        ],
       [0.01152044, 0.02600314, 0.20201097, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.2099885 , 0.20656347, 0.22703551, ..., 0.        , 0.        ,
        0.        ],
       [0.17811752, 0.17521232, 0.19257722, ..., 0.        , 0.        ,
        0.        ],
       [0.11972951, 0.11777665, 0.12944923, ..., 0.        , 0.        ,
        0.        ]])

In [11]:
peak_count

11.0

In [24]:
pd.to_pickle(peak_format, "trained.pkl")

In [12]:
a = pd.read_pickle("synthetic_data/peaks_synthetic_formatted.pkl")

In [13]:
cutted_gt = a[train_len:]

In [14]:
pd.to_pickle(cutted_gt, 'gt_peaks_val_set.pkl')

In [129]:
angle_pairs = pd.read_pickle("synthetic_data/angle_pairs.pkl")
evals = np.tile([0.0017, 0.0002, 0.0002], (len(angle_pairs), 1))

In [130]:
def cartesian_to_spherical(x, y, z):
    # Calculate phi (azimuthal angle)
    phi = math.atan2(y, x)
    
    # Calculate theta (polar angle)
    r_xy = math.sqrt(x**2 + y**2)
    theta = math.atan2(r_xy, z)
    
    # Convert angles to degrees
    phi_deg = math.degrees(phi)
    theta_deg = math.degrees(theta)
    
    return phi_deg,theta_deg

In [131]:
def angles_fractions(gt):
    angles = []
    fractions = []
    for i in range(len(gt)//3):
        if gt[3*i] != 0.0:
            phi, theta = cartesian_to_spherical(gt[3*i], gt[3*i+1], gt[3*i+2])
            angles.append((theta, phi))
            fractions.append(gt[3*i]**2 + gt[3*i+1]**2 + gt[3*i+2]**2)
    fractions = fractions / sum(fractions)
    return angles, fractions


In [133]:
sphere2 = get_sphere('repulsion724').subdivide(1)

random.seed(5)

for i in range(15):
    
    random_number = random.randint(0, len(F_pred_np))
    _, peak_count = detect_peaks(F_pred_np[random_number].reshape(1,-1), 0.5, 25)
    gt_peak_count = np.count_nonzero(gt_white_masked[random_number]) / 3
    odf = multi_tensor_odf(sphere2.vertices, evals, angles=angle_pairs,
                              fractions=F_pred_np[random_number])
    
    gt_angles, gt_fractions = angles_fractions(gt_white_masked[random_number])
    odf_gt = multi_tensor_odf(sphere2.vertices, evals, angles=gt_angles,
                              fractions=gt_fractions)
    
    from dipy.viz import window, actor
    
    # Enables/disables interactive visualization
    interactive = False
    
    
    scene = window.Scene()
    
    odfs = np.vstack((odf, odf_gt))[:, None, None]
    odf_actor = actor.odf_slicer(odfs, sphere=sphere2, scale=0.5, colormap='plasma')
    
    odf_actor.display(y=0)
    odf_actor.RotateX(90)
    scene.add(odf_actor)
    window.record(scene, out_path='trained_model/hardi_results/threshold_3_res_' + str(random_number) + 'peak_count_' + str(peak_count) + '_gt_peak_count_' + str(gt_peak_count) + '.png', size=(300, 300))
    if interactive:
        window.show(scene)